In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import xml.etree.ElementTree as ET
import requests

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(('localhost', 8000), requestHandler=RequestHandler)
server.register_introspection_functions()

# Initialize an empty XML database with a root element
with open('database.xml', 'w') as file:
    file.write('<root></root>')

# Parse the XML database file
xml_db = ET.parse('database.xml')

def process_client_input(topic, text, timestamp):
    global xml_db
    root = xml_db.getroot()
    topic_exists = False
    note = None

    # Check if the topic already exists
    for existing_note in root.findall(".//note[@topic='{}']".format(topic)):
        topic_exists = True
        note = existing_note
        break

    # If the topic doesn't exist, create a new note
    if not topic_exists:
        note = ET.SubElement(root, 'note', attrib={'topic': topic})

    # Add data and timestamp to the note
    new_note = ET.SubElement(note, 'data')
    new_note.text = text

    new_timestamp = ET.SubElement(note, 'timestamp')
    new_timestamp.text = timestamp

    # Save data to XML file
    xml_db.write('database.xml')

    return "Data saved successfully!"

def get_xml_contents(topic):
    global xml_db
    root = xml_db.getroot()
    notes = root.findall(".//note[@topic='{}']".format(topic))
    result = []

    for note in notes:
        data = note.find('data').text
        timestamp = note.find('timestamp').text
        result.append({'data': data, 'timestamp': timestamp})

    return result

def query_wikipedia(search_term):
    # Construct Wikipedia API URL
    wikipedia_url = f"https://en.wikipedia.org/w/api.php"
    params = {
        'action': 'opensearch',
        'search': search_term,
        'format': 'json'
    }

    # Make a request to Wikipedia API
    response = requests.get(wikipedia_url, params=params)
    data = response.json()

    if data[3]:
        # If Wikipedia article found, add relevant information to user-submitted topic
        article_link = data[3][0]
        return f"Link to Wikipedia article: {article_link}"

    return "No Wikipedia article found for the given search term."

# Register functions for XML-RPC
server.register_function(process_client_input, 'process_client_input')
server.register_function(get_xml_contents, 'get_xml_contents')
server.register_function(query_wikipedia, 'query_wikipedia')

# Start the server
server.serve_forever()
